In [2]:
import itertools
import numpy as np
from multiprocessing.dummy import Pool
import os.path
import mpu
import time
import sys
sys.path.append('../')
from card import Card

In [3]:
if os.path.isfile('SevenCardHands.npy'):
    SevenCardHands = np.load('SevenCardHands.npy')
else:
    SevenCardHands = np.array(list(itertools.combinations(range(32),7)))
    np.save('SevenCardHands.npy', SevenCardHands)
print(SevenCardHands.shape)

(3365856, 7)


In [4]:
if os.path.isfile('FiveCardHands.npy'):
    FiveCardHands = np.load('FiveCardHands.npy')
else:
    FiveCardHands = np.array(list(itertools.combinations(range(32),5)))
    np.save('FiveCardHands.npy', FiveCardHands)
print(FiveCardHands.shape)

(201376, 5)


In [7]:
def score_hand(hand):
    flush = hand.sum(axis=0).sum(axis=1)
    is_flush = np.max(flush) >= 5
    card_values = hand.sum(axis=0).sum(axis=0)
    count = 0
    hand_high = 8
    is_stright = False
    stright_high = 5
    for i, s in enumerate(card_values): 
        if s >= 1:
            count += 1
            hand_high = i
            if count >=5:
                stright_high = i
                is_stright = True
        else:
            count = 0
    if is_stright and is_flush:
        return 8, stright_high, hand_high    
    is_four_of_a_kind = np.isin(card_values, 4)
    if np.sum(is_four_of_a_kind):
        return 7, np.argwhere(is_four_of_a_kind==True)[0][0], hand_high
    is_three_of_a_kind = np.isin(card_values, 3)
    is_pair = np.isin(card_values, 2)
    if np.sum(is_pair) and np.sum(is_three_of_a_kind):
        return 6, np.argwhere(is_three_of_a_kind==True)[0][0], hand_high
    if is_flush:
        return 5, np.argwhere(hand.sum(axis=0)[np.argmax(flush)]==1)[-1][0], hand_high
    if is_stright:
        return 4, stright_high, hand_high
    if np.sum(is_three_of_a_kind):
        return 3, np.argwhere(is_three_of_a_kind==True)[0][0], hand_high
    if np.sum(is_pair) >= 2:
        return 2, np.argwhere(is_pair==True)[1][0], hand_high
    if np.sum(is_pair):
        return 1, np.argwhere(is_pair==True)[0][0], hand_high
    return 0, 0, hand_high

In [11]:
HandScores = {}
def run(k):
    hand = np.zeros((7, 32), dtype=np.int8)
    for i in range(7):
        hand[i,SevenCardHands[k,i]] = 1
    hand = hand.reshape(7,4,8)
    HandScores[SevenCardHands[k,:].tobytes()] = score_hand(hand)
p = Pool(10)
p.map(run, range(SevenCardHands.shape[0]))
p.close()
p.join()
mpu.io.write('filename.pickle', HandScores)

In [8]:
HandScores = mpu.io.read('filename.pickle')

In [12]:
for hand in SevenCardHands:
    if np.all(np.in1d(FiveCardHands[0], hand)):
        print(HandScores[hand.tobytes])

[0 1 2 3 4 5 6]


In [5]:
'''
scores = {}
num_cards = 7
iters = 1
for hand in FiveCardHands[0:260]:
    histo = np.zeros((9))
    deck = np.arange(52)
    deck = np.delete(deck, hand)
    for i in range(iters):
        np.random.shuffle(deck)
        cards = np.zeros((num_cards,52))
        for i, c in enumerate(hand):
            cards[i,c] = 1
        for i in range(len(hand),num_cards):
            cards[i,deck[i]] = 1
        cards = cards.reshape(num_cards,4,13)
        histo[score_hand(cards)[0:1]] += 1
    score = 0
    for i in range(9):
        score += (i+1)*histo[i]/iters
    scores[hand.tobytes()] = score
np.save('5_card_values_'+str(iters)+'.npy', scores)
scores
'''

"\nscores = {}\nnum_cards = 7\niters = 1\nfor hand in FiveCardHands[0:260]:\n    histo = np.zeros((9))\n    deck = np.arange(52)\n    deck = np.delete(deck, hand)\n    for i in range(iters):\n        np.random.shuffle(deck)\n        cards = np.zeros((num_cards,52))\n        for i, c in enumerate(hand):\n            cards[i,c] = 1\n        for i in range(len(hand),num_cards):\n            cards[i,deck[i]] = 1\n        cards = cards.reshape(num_cards,4,13)\n        histo[score_hand(cards)[0:1]] += 1\n    score = 0\n    for i in range(9):\n        score += (i+1)*histo[i]/iters\n    scores[hand.tobytes()] = score\nnp.save('5_card_values_'+str(iters)+'.npy', scores)\nscores\n"

In [9]:
scores = {}
num_cards = 7
iters = 100

def score_hands(index):
    hand = FiveCardHands[int(index)]
    histo = np.zeros((9))
    deck = np.arange(52)
    deck = np.delete(deck, hand)
    for i in range(iters):
        np.random.shuffle(deck)
        cards = np.zeros((num_cards,52))
        for k, c in enumerate(hand):
            cards[k,c] = 1
        for k in range(len(hand),num_cards):
            cards[k,deck[k]] = 1
        cards = cards.reshape(num_cards,4,13)
        histo[score_hand(cards)[0:1]] += 1
    score = 0
    for i in range(9):
        score += (i+1)*histo[i]/iters
    scores[hand.tobytes()] = score

p = Pool(10)
p.map(score_hands, range(FiveCardHands.shape[0]))
p.close()
p.join()
np.save('5_card_values_'+str(iters)+'.npy', scores)

In [134]:
FiveCardHands.shape[0]

2598960